In [1]:
import pandas as pd
from geopy.distance import great_circle

# Load datasets
walmart_geocoded = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/walmart_geocoded.csv')
zips_geocoded = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/zips_geocoded.csv')
distance_data = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/distance.csv')

# Function to find great circle distance
def get_great_circle_distance(zip_coords, store_coords):
    return great_circle(zip_coords, store_coords).miles

# Prepare the output file
output_file = '/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/processed/time_distance.csv'
with open(output_file, 'w') as f:
    f.write('zip,walmarts_pot_10\n')

# Process each zip code
for index, row in distance_data.iterrows():
    zip_code = row['zip']
    nearby_stores = str(row['nearby_walmarts']).split(',')
    stores_10_under = []

    for store_number in nearby_stores:
        store_number = store_number.strip()
        if store_number.isdigit() and int(store_number) in walmart_geocoded['Store #'].values:
            store_coords = walmart_geocoded[walmart_geocoded['Store #'] == int(store_number)][['latitude', 'longitude']].iloc[0]
            zip_coords = zips_geocoded[zips_geocoded['zip'] == zip_code][['latitude', 'longitude']].iloc[0]
            distance = get_great_circle_distance((zip_coords['latitude'], zip_coords['longitude']), (store_coords['latitude'], store_coords['longitude']))

            if distance <= 10:  # 10 miles or less
                stores_10_under.append(store_number)

    # Write to the output file
    with open(output_file, 'a') as f:
        line = f"{zip_code},{' '.join(stores_10_under)}\n"
        f.write(line)